# High-throughput screening for novel materials? - Exercises

In this notebook, you find some exercises related with how to perform a high-throughput screening for light harvesting materials.

Use the below cell to read the database into memory.

In [ ]:
%matplotlib inline
from ase.db import connect
con = connect('cubic_perovskites.db')

## Exercise 1
From the cubic_perovsites.db database, identify the oxides that can be used for one photon water splitting device.

You can take inpiration from the CMR database, https://cmr.fysik.dtu.dk/cubic_perovskites/cubic_perovskites.html#cubic-perovskites.

## Exercise 2
Use AgNbO$_3$, and calculate the convex hull and Pourbaix diagram using data from the database.
See this tutorial for inspiration: https://wiki.fysik.dtu.dk/ase/ase/phasediagram/phasediagram.html

For the Pourbaix diagram, remember to give the reference energies with respect to the standard states.

Is the material on the convex hull? If not, why is it still a candidate?

## Exercise 3
One of the possible solutions to instability in water is to add a transparent protective layer with appropriate position of the bands and stable in water. Find the possible transparent layers in the oxo-perovskites for AgNbO$_3$. (hint: consider only the oxygen evolution band edge and, for simplicity, consider only the indirect gap and discard the calculation of Pourbax diagrams)

Did you find any good candidate? If not, try with another material.

## Exercise 4
Relax, calculate the band gap and band structure at the GLLB-SC level of a possible new candidate structure: CaTiO$_3$ and compare it with SrTiO$_3$. Take inspiration from the exercises of the previous lecture and https://wiki.fysik.dtu.dk/gpaw/tutorials/band_gap/band_gap.html and https://wiki.fysik.dtu.dk/gpaw/exercises/band_structure/bands.html

Calculate the band structure and band gap of both structures.

The GLLB functional contains a bug, where it is unable to parallelize over bands, i.e. if you use plane-wave mode. You can avoid this error by using the following keyword in your GPAW calculator
```python
parallel={'band': 1}
```

As we will need to plot the band structure later, make sure that you save all of the wave functions for your fixed density calculation, i.e. use `mode='all'` when writing your wave functions:
```python
calc.write('myfile.gpw', mode='all')
```
We have provided you with a function to create a band structure object, which  you can plot using something along the lines of
```python
gap, bs = get_band_structure('myfile.gpw')
print('Gap:', gap)
bs.plot()
```

In [ ]:
%%writefile relax.py


In [ ]:
from gpaw import GPAW
from ase.dft.bandgap import bandgap

def get_band_structure(gpw):
    '''Get a band structure object from a .gpw file.
    Note that wave functions need to be saved to the .gpw file to work.
    I.e. in your calculation, do
    calc.write('myfile.gpw', mode='all')
    
    Then plot the band structure with something like
    
    gap, bs = get_band_structure('myfile.gpw')
    print('Gap:', gap)
    bs.plot()
    '''
    calc = GPAW(gpw, txt=None)
    bs = calc.band_structure()
    gap = bandgap(calc)[0]
    xc = calc.get_xc_functional()
    ef = calc.get_fermi_level()
    if xc == 'GLLBSC':
        # Get GLLBsc band gap
        response = calc.hamiltonian.xc.xcs['RESPONSE']
        response.calculate_delta_xc()
        EKs, Dxc = response.calculate_delta_xc_perturbation()
        homo, lumo = calc.get_homo_lumo()
        # Modify energies
        bs.energies[bs.energies >= ef] += Dxc
        gap += Dxc
        
    bs.energies -= ef  # Center energies at Fermi level
    bs.reference -= ef
    return gap, bs